In [43]:
import os
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model 
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.model_selection import train_test_split

In [3]:
H = 512
W = 512

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
def build_model(input_shape, num_classes=196):
    inputs = Input(input_shape)

    backbone = MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        alpha=1.0
    )

    #backbone.summary()

    X = backbone.output
    #y = backbone.get_layer("block_13_expand_relu").output
    X = Conv2D(256, kernel_size=1, padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = GlobalAveragePooling2D()(X)
    X = Dropout(0.5)(X)
    X = Dense(4, activation="sigmoid")(X)

    model = Model(inputs, X)
    
    return model

In [31]:
if __name__=="__main__":
    input_shape = (512, 512, 3)

    model = build_model(input_shape)

    model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_9[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']      

In [39]:
def load_data(path, split=0.1):
    images = []
    bboxes = []

    df = pd.read_csv(os.path.join(path, "bbox.csv"))
    for index, row in df.iterrows():
        #print(index, row)
        name = row["name"]
        X1 = int(row["x1"])
        y1 = int(row["y1"])
        X2 = int(row["x2"])
        y2 = int(row["y2"])

        image = os.path.join(path, "images", name)
        bbox = [X1, y1, X2, y2]

        images.append(image)
        bboxes.append(bbox)

    #print(len(images), len(bboxes))
    split_size = int(len(images) * split)

    X_train, X_val = train_test_split(images, test_size=split_size, random_state=42)
    y_train, y_val = train_test_split(bboxes, test_size=split_size, random_state=42)

    X_train, X_test = train_test_split(X_train, test_size=split_size, random_state=42)
    y_train, y_test = train_test_split(y_train, test_size=split_size, random_state=42)

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)
    

In [11]:
def read_image_bbox(path, bbox):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)

    h, w, _ = image.shape
    image = cv2.resize(image, (W, H))
    image = (image - 127.5) / 127.5
    image = image.astype(np.float32)

    X1, y1, X2, y2 = bbox
    norm_x1 = float(X1 / w)
    norm_y1 = float(y1 / h)
    norm_x2 = float(X2 / w)
    norm_y2 = float(y2 / h)

    norm_bbox = np.array([norm_x1, norm_y1, norm_x2, norm_y2], dtype=np.float32)
    return image, norm_bbox

In [13]:
def parse(x, y):
    x, y = tf.numpy_function(read_image_bbox, [x, y], [tf.float32, tf.float32])
    x.set_shape([W, H, 3])
    y.set_shape([4])
    return x, y

In [15]:
def tf_dataset(images, bboxes, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((images, bboxes))
    ds = ds.map(parse).batch(batch).prefetch(10)
    return ds

In [17]:
data_path = r"E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)"
dataset_path = r"E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)\images"
if __name__=="__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(data_path + "\\files")

    data = os.path.join(data_path, "\\files")

    batch_size = 8
    lr = 1e-4
    num_epochs = 100
    
    model_path = os.path.join(data_path + "files" + "human_detection_100_epochs.h5")
    csv_path = os.path.join(data_path + "files" + "log.csv")

    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data(data_path)
    print(f"Train: {len(X_train)}, {len(y_train)}")
    print(f"Val: {len(X_val)}, {len(y_val)}")
    print(f"Test: {len(X_test)}, {len(y_test)}")

    train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
    val_dataset = tf_dataset(X_val, y_val, batch=batch_size)

    #for x, y in train_dataset:
        #print(x.shape, y.shape)
        #idx = 3
        #image = (x[idx].numpy() + 1) * 127.5
        #X1 = int(y[idx][0] * image.shape[1])
        #y1 = int(y[idx][1] * image.shape[0])
        #X2 = int(y[idx][2] * image.shape[1]) 
        #y2 = int(y[idx][3] * image.shape[0])

        #image = cv2.rectangle(image, (X1, y1), (X2, y2), (0, 255, 0), 5)
        #cv2.imwrite(data_path + "\\1.png", image)
        
        #break

    model = build_model((H, W, 3))
    model.compile(
        loss = "binary_crossentropy",
        optimizer=Adam(learning_rate=lr)
    )

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_delta=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
    ]

Train: 679, 679
Val: 84, 84
Test: 84, 84




In [ ]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data = val_dataset,
    callbacks = callbacks
)

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 0.6685
Epoch 1: val_loss improved from inf to 0.58329, saving model to E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)fileshuman_detection_100_epochs.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 219s 2s/step - loss: 0.6685 - val_loss: 0.5833 - lr: 1.0000e-04
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.4902
Epoch 2: val_loss improved from 0.58329 to 0.47854, saving model to E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)fileshuman_detection_100_epochs.h5
85/85 [==============================] - 140s 2s/step - loss: 0.4902 - val_loss: 0.4785 - lr: 1.0000e-04
Epoch 3/100
85/85 [==============================] - ETA: 0s - loss: 0.4620
Epoch 3: val_loss improved from 0.47854 to 0.46504, saving model to E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)fileshuman_detection_100_epochs.h5
85/85 [==============================] - 141s 2s/step - loss: 0.4620 - val_loss: 0.4650 - lr: 1.0000e-04
Epoch 4/100
85/85 [==============================] - ETA: 0s - loss: 0.4513
Epoch 4: val_loss improved from 0.46504 to 0.46328, saving model to E:\python\segmentation\Computer Vision\UNET\data\ob

In [ ]:
model.save("Human_detection_100_epochs.h5")

In [25]:
def call_iou(y_true, y_pred):
    X1 = max(y_true[0], y_pred[0])
    y1 = max(y_true[1], y_pred[1])
    X2 = max(y_true[2], y_pred[2])
    y2 = max(y_true[3], y_pred[3])

    intersection_area = max(0, X2-X1+1) * max(0, y2 - y1+1)

    true_area = (y_true[2] - y_true[0] + 1) * (y_true[3] - y_true[1] + 1)
    bbox_area = (y_pred[2] - y_pred[0] + 1) * (y_pred[3] - y_pred[1] + 1)

    iou = intersection_area / float(true_area + bbox_area - intersection_area)
    return iou

In [47]:
data_path = r"E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)"
dataset_path = r"E:\python\segmentation\Computer Vision\UNET\data\objectarchive (2)"
model_path = r"E:\python\segmentation\Computer Vision\UNET\data"
if __name__=="__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(data_path + "\\results")
    trained_model = load_model(os.path.join(model_path, "objectarchive (2)fileshuman_detection_100_epochs.h5"), compile=False)

    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data(dataset_path)
    print(f"Train : {len(X_train)} - {len(y_train)}")
    print(f"Val : {len(X_val)} - {len(y_val)}")
    print(f"Test : {len(X_test)} - {len(y_test)}")

    mean_iou = []
    for image, true_bbox in tqdm(zip(X_test, y_test), total=len(X_test)):
        name = image.split("\\")[-1].split(".")[0]
        print(name)

        image = cv2.imread(image, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (W, H))
        x = (image - 127.5) / 127.5
        #x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
    
        X1_true, y1_true, X2_true, y2_true = true_bbox
                                
        pred_bbox = trained_model.predict(x, verbose=0)[0]
        
        pred_x1 = int(pred_bbox[0] * image.shape[1])
        pred_y1 = int(pred_bbox[1] * image.shape[0])
        pred_x2 = int(pred_bbox[2] * image.shape[1])
        pred_y2 = int(pred_bbox[3] * image.shape[0])

        iou = call_iou(true_bbox, [pred_x1, pred_y1, pred_x2, pred_y2])
        mean_iou.append(iou)

        image = cv2.rectangle(image, (X1_true, y1_true), (X2_true, y2_true), (0, 255, 0), 10)
        image = cv2.rectangle(image, (pred_x1, pred_y1), (pred_x2, pred_y2), (0, 0, 255), 10)

        x = int(image.shape[1] * 0.05)
        y = int(image.shape[0] * 0.05)
        font_size = int(image.shape[0] * 0.001)
        cv2.putText(image, f"IoU: {iou:.4f}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 255, 0), 3)

        cv2.imwrite(data_path + f"\\files\\{name}.png", image)

    score = np.mean(mean_iou, axis=0)
    print(f"Mean IoU: {score:.4f}")

Train : 679 - 679
Val : 84 - 84
Test : 84 - 84


  0%|                                                                                           | 0/84 [00:00<?, ?it/s]

photo-1466709270977-7b387d9d3471


  1%|▉                                                                                  | 1/84 [00:07<10:04,  7.29s/it]

photo-1518182457238-aacf9536971d


...

photo-1584911883494-14280ff783e4


 99%|█████████████████████████████████████████████████████████████████████████████████ | 83/84 [00:53<00:00,  1.70it/s]

photo-1606208397452-29faa5b695f5


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [00:53<00:00,  1.56it/s]

Mean IoU: 25.1855
